## _*Using QISKit ACQUA for maxcut problems*_

This QISKit ACQUA Optimization notebook demonstrates how to use the VQE quantum algorithm to compute the max cut of a given graph. 

The problem is defined as follows. Given a graph $G = (V,E)$ with weights $w_{ij}$ on the edges, we are looking for a subset $S \subseteq V$ such that $\sum_{(i,j) \in E : i \in S, j \not\in S} w_{ij}$ is maximized.

The graph provided as an input is used first to generate an Ising Hamiltonian, which is then passed as an input to VQE.  As a reference, this notebook also computes the max cut using the Exact Eigensolver classical algorithm and the solver embedded in the commercial non-quantum IBM CPLEX product (if it is available in the system and the user has followed the necessary configuration steps in order for QISKit ACQUA to find it).  Please refer to the QISKit ACQUA Optimization documentation for installation and configuration details.

In [ ]:
from qiskit_acqua import Operator, run_algorithm, get_algorithm_instance
from qiskit_acqua.input import get_input_instance
from qiskit_acqua.ising import maxcut
import numpy as np

Here an Operator instance is created for our Hamiltonian. In this case the paulis are from an Ising Hamiltonian of the max cut problem. We load a small instance of the maxcut problem.

In [2]:
w = maxcut.parse_gset_format('sample.maxcut')
qubitOp, offset = maxcut.get_maxcut_qubitops(w)
algo_input = get_input_instance('EnergyInput')
algo_input.qubit_op = qubitOp

We also offer a function to generate a random graph as a input.

In [3]:
if True:
    np.random.seed(8123179)
    w = maxcut.random_graph(4, edge_prob=0.5, weight_range=10)
    qubitOp, offset = maxcut.get_maxcut_qubitops(w)
    algo_input.qubit_op = qubitOp
print(w)

[[ 0.  8. -9.  0.]
 [ 8.  0.  7.  9.]
 [-9.  7.  0. -8.]
 [ 0.  9. -8.  0.]]


In [4]:
to_be_tested_algos = ['ExactEigensolver', 'CPLEX', 'VQE']
operational_algos = []
for algo in to_be_tested_algos:
    try:
        get_algorithm_instance(algo)
        operational_algos.append(algo)
    except:
        print("{} is unavailable, please check your setting.".format(algo))
print(operational_algos)

['ExactEigensolver', 'CPLEX', 'VQE']


We can now use the Operator without regard to how it was created. First we need to prepare the configuration params to invoke the algorithm. Here we will use the ExactEigensolver first to return the smallest eigenvalue. Backend is not required since this is computed classically not using quantum computation. We then add in the qubitOp Operator in dictionary format. Now the complete params can be passed to the algorithm and run. The result is a dictionary.

In [5]:
if 'ExactEigensolver' not in operational_algos:
    print("ExactEigensolver is not in operational algorithms.")
else:
    algorithm_cfg = {
        'name': 'ExactEigensolver',
    }

    params = {
        'problem': {'name': 'ising'},
        'algorithm': algorithm_cfg
    }
    result = run_algorithm(params,algo_input)
    # print('objective function:', maxcut.maxcut_obj(result, offset))
    x = maxcut.sample_most_likely(len(w), result['eigvecs'][0])
    print('energy:', result['energy'])
    print('maxcut objective:', result['energy'] + offset)
    print('solution:', maxcut.get_graph_solution(x))
    print('solution objective:', maxcut.maxcut_value(x, w))

energy: -20.5
maxcut objective: -24.0
solution: [1. 0. 1. 1.]
solution objective: 24.0


We change the configuration parameters to solve it with the CPLEX backend. The CPLEX backend can deal with a particular type of Hamiltonian called Ising Hamiltonian, which consists of only Pauli Z at most second order and often for combinatorial optimization problems that can be formulated as quadratic unconstrained binary optimization problems, such as the max-cut problem.

Note that for a maxcut problem, since we are computing a bipartition of the graph, every binary vector $x$ and its complement (i.e., the vector $y$ such that $y_j = 1 - x_j$ for all $j$) represent exactly the same solution, and will have the same objective function value. Different solution methods may return solutions that look different, but in fact have the same objective function value.

In [6]:
if 'CPLEX' not in operational_algos:
    print("CPLEX is not in operational algorithms.")
else:
    algorithm_cfg = {
        'name': 'CPLEX',
        'display': 0
    }

    params = {
        'problem': {'name': 'ising'},
        'algorithm': algorithm_cfg
    }

    result = run_algorithm(params, algo_input)

    x_dict = result['x_sol']
    print('energy:', result['energy'])
    print('time:', result['time'])
    print('maxcut objective:', result['energy'] + offset)
    x = np.array([x_dict[i] for i in sorted(x_dict.keys())])
    print('solution:', maxcut.get_graph_solution(x))
    print('solution objective:', maxcut.maxcut_value(x, w))

CPXPARAM_TimeLimit                               600
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1
CPXPARAM_MIP_Tolerances_Integrality              0
CPXPARAM_MIP_Display                             0
energy: -20.5
time: 0.007260107027832419
maxcut objective: -24.0
solution: [1 0 1 1]
solution objective: 24.0


Now we want VQE and so change it and add its other configuration parameters. VQE also needs and optimizer and variational form. While we can omit them from the dictionary, such that defaults are used, here we specify them explicitly so we can set their parameters as we desire.

In [8]:
if 'VQE' not in operational_algos:
    print("VQE is not in operational algorithms.")
else:
    algorithm_cfg = {
        'name': 'VQE',
        'operator_mode': 'matrix'
    }

    optimizer_cfg = {
        'name': 'L_BFGS_B',
        'maxfun': 6000
    }

    var_form_cfg = {
        'name': 'RYRZ',
        'depth': 3,
        'entanglement': 'linear'
    }

    params = {
        'problem': {'name': 'ising'},
        'algorithm': algorithm_cfg,
        'optimizer': optimizer_cfg,
        'variational_form': var_form_cfg,
        'backend': {'name': 'local_statevector_simulator'}
    }

    result = run_algorithm(params,algo_input)

    x = maxcut.sample_most_likely(len(w), result['eigvecs'][0])
    print('energy:', result['energy'])
    print('time:', result['eval_time'])
    print('maxcut objective:', result['energy'] + offset)
    print('solution:', maxcut.get_graph_solution(x))
    print('solution objective:', maxcut.maxcut_value(x, w))

energy: -20.499999999999712
time: 34.83333230018616
maxcut objective: -23.999999999999712
solution: [1. 0. 1. 1.]
solution objective: 24.0
